In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import softops

In [ ]:
with tf.Graph().as_default():
    x = tf.placeholder(tf.float32, shape=(None), name="x")
    
    for scale in [1, 10, 100]:
        for y1, y2 in [(tf.sin(10*x), 3*x**2), (10+3*x, 3)]:
            y = softops.threshold(x, y1, y2, softness=0.2, soft_value=False, soft_grad=True)
            y_soft = softops.threshold(x, y1, y2, softness=.2, soft_value=True, soft_grad=True)


            #y = softops.positive(x, softness=1, soft_value=False)
            #y_soft = softops.positive(x, softness=1, soft_value=True)

            #y = softops.gt(x, scale/2, percent=True, softness=0.3, soft_value=False)
            #y_soft = softops.gt(x, scale/2, percent=False, softness=0.3, soft_value=True)

            #y = (x + 5)*(1-y) + 2*(y)
            #y_soft = (x + 5)*(1-y_soft) + 2*(y_soft)

            gradient = tf.gradients(y, [x])

            with tf.Session() as session:
                data_x = np.linspace(-scale, scale, 1001)
                data_y, data_y_soft, data_grad = session.run([y, y_soft, gradient], feed_dict={x: data_x})

                #print(data_y)
                #print(data_grad)
                plt.grid(True)
                plt.plot(
                    data_x, data_y, 'r-', 
                    data_x, data_y_soft, 'g-', 
                    data_x, data_grad[0], 'b-'
                )
                plt.show()